In [0]:
import numpy as np

In [4]:
###build model, inception v3
###7 labels
from keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(include_top = False, weights='imagenet', input_shape=(132,132,3), classes=7)
base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 3s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 132, 132, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 65, 65, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 65, 65, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 65, 65, 32)   0           batch_

In [0]:
for layer in base_model.layers:
  layer.trainable = False

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.losses import categorical_crossentropy
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [0]:
nb_features = 64
batch_size = 128
nb_class = 7
nb_epochs = 100

In [8]:
inception = Sequential()

inception.add(base_model)
inception.add(BatchNormalization())
inception.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
inception.add(Dropout(0.5))
inception.add(Flatten())

inception.add(Dense(2*2*2*nb_features, activation='relu'))
inception.add(Dropout(0.4))
inception.add(Dense(2*2*nb_features, activation='relu'))
inception.add(Dropout(0.4))
inception.add(Dense(2*nb_features, activation='relu'))
inception.add(Dropout(0.5))

inception.add(Dense(nb_class, activation='softmax'))

inception.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2, 2, 2048)        21802784  
_________________________________________________________________
batch_normalization_95 (Batc (None, 2, 2, 2048)        8192      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 1, 1, 2048)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
___________________________

In [0]:
MODEL_PATH = './model/inception_nbclass7.h5'

In [0]:
inception.compile(loss=categorical_crossentropy, 
                  optimizer=Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-8), 
                  metrics = ['accuracy'])

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', 
                               factor=-np.log(0.9),
                               patience=4,
                               verbose = 1)

tensorboard = TensorBoard(log_dir="./logs")

early_stopper = EarlyStopping(monitor='val_loss',min_delta=0 ,patience=4, verbose = 1)

checkpointer = ModelCheckpoint(MODEL_PATH, monitor='val_loss', verbose=1, save_best_only=True)

In [0]:
inception.fit(X_train, Y_train,
              batch_size=BATCHSIZE,
              epochs=nb_epochs,
              verbose=1,
              validation_data=(X_valid, Y_valid),
              shuffle=True,
              callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])